In [1]:
import pandas as pd
import numpy as np 
import psycopg2
import cufflinks
from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
connection = psycopg2.connect(dbname='papahere', user='postgres', password='ncr*123')
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# candlesize =60
query = '''
select 
to_timestamp(floor((extract('epoch' from a.exchange_timestamp) / 60 )) * 60) AT TIME ZONE 'UTC' as candle,
tradingsymbol,
(array_agg (last_price order by a.last_trade_time asc ))[1] o,
(array_agg (last_price order by a.last_trade_time desc ))[1] c,
max (a.last_price) h,
min (a.last_price) l,
avg(a.total_buy_quantity) total_buy_quantity, 
avg(a.total_sell_quantity)total_sell_quantity,
(array_agg (volume_traded order by a.last_trade_time desc ))[1] volume,
(array_agg (oi order by a.last_trade_time desc ))[1] oi,
avg(a.buy_bid)buy_bid,
avg(a.sell_bid)sell_bid
from archive.ticks a where a.tradingsymbol = 'BANKNIFTY2231734000PE' and date(exchange_timestamp) = '15mar2022'
-- and extract (hour from a.last_trade_time) between 11 and 12
group by candle , a.tradingsymbol
order by candle 
'''

In [3]:
data =pd.read_sql(query, connection)

C:\Users\ssmgf\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



In [4]:
def hacandles(data):
    dataha=data[[ 'o', 'c', 'h', 'l']].copy()
    for i in range (1,len(dataha)):
        dataha.iat[i,1] = sum(dataha.iloc[i])/4
        dataha.iat[i,0] = sum(dataha.iloc[i-1,:2])/2
        dataha.iat[i,2] = dataha.iloc[i,[0,1,2]].max() 
        dataha.iat[i,-1] = dataha.iloc[i,[0,1,-1]].min() 
        dataha.columns = [ 'hao', 'hac', 'hah', 'hal']
        dataha.index=data.candle
    return dataha



In [9]:
dataha = hacandles(data)

In [50]:
fig= data.iplot(asFigure=True,y=['oi'],secondary_y=['volume_avg'] ,x=['candle'] )
fig.add_candlestick(x= data.candle,
                    open=dataha.hao,
                    close=dataha.hac,
                    high=dataha.hah,
                    low=dataha.hal,yaxis='y3')
fig.update_layout(dict(yaxis3=dict(
        title="yaxis3 title",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    )))
fig.layout.xaxis.rangeselector.visible=False
fig.layout.xaxis.rangeslider.visible=False
fig.write_html('check.html')


In [61]:
v=data.volume.rolling(window=15,min_periods=1).mean()
v1=v - v.shift(14)
v1 = v1/v.shift(14)
data['volume_avg']=v1
# v1=v1/


In [59]:
v1

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
365    0.049500
366    0.049056
367    0.048531
368    0.047816
369    0.046965
Name: volume, Length: 370, dtype: float64

In [60]:
px.line(v1)